CLUSTERING the neighborhood of toranto

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
source=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup=BeautifulSoup(source,'lxml')
table_soup=soup.table.tbody.find_all('tr')
table_soup.remove(table_soup[0])
pc=list()
br=list()
ne=list()
for row in table_soup:
    try:
        pc.append(row.find_all('td')[0].text)
        br.append(row.find_all('td')[1].text)
        ne.append(row.find_all('td')[2].text.strip())
    except Exception as e:
        pass
toranto_df=pd.DataFrame({'Postalcode':pc,'Borough':br,'Neighborhood':ne})

1. Geting the html page from the web and striping the table information into the arrays
  and storing the arrays data into data frame

In [3]:
toranto_df=toranto_df.drop(toranto_df[toranto_df['Borough']=='Not assigned'].index)

Removing the cells which are having 'Not assigned' in "Borough"

In [4]:
toranto_df.loc[toranto_df['Neighborhood']=='Not assigned','Neighborhood']=toranto_df.loc[toranto_df['Neighborhood']=='Not assigned','Borough']

Grouping data by postal code

In [5]:
toranto_gr=toranto_df.groupby("Postalcode")

In [6]:
toranto_gr

Joining the Neighborhoods acc. to postalcode

In [7]:
df=pd.DataFrame(columns=['Postalcode','Borough','Neighborhood'])
i=0
for grp in pd.unique(toranto_df["Postalcode"]):
    ni=list()
    grp_df=toranto_gr.get_group(grp)
    ni=grp_df['Neighborhood'].values
    #temp_df=pd.DataFrame({'Postalcode':[grp],'Borough':pd.unique(grp_df['Borough']),'Neighborhood':','.join(ni)})
    #print(temp_df)
    df.loc[i]=[grp,pd.unique(grp_df['Borough'])[0],','.join(ni)]
    i=i+1
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [8]:
pip install geocoder # import geocoder


The following command must be run outside of the IPython shell:

    $ pip install geocoder # import geocoder

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more informations on how to install packages:

    https://docs.python.org/3/installing/


In [9]:
!wget -o 'toranto_geodata.csv' https://cocl.us/Geospatial_data
data=pd.read_csv('https://cocl.us/Geospatial_data')
data.head()

'wget' is not recognized as an internal or external command,
operable program or batch file.


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
lat_arr=list()
lon_arr=list()
for postal_code in df['Postalcode'].values:
    lat_arr.append(data.loc[data['Postal Code']==postal_code,'Latitude'].values[0])
    lon_arr.append(data.loc[data['Postal Code']==postal_code,'Longitude'].values[0])
df['Latitude']=lat_arr
df['Longitude']=lon_arr
df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


Geting longitude and latitude from .csv file

In [12]:
df.shape

(103, 5)

imorting packeges neede for the maps

In [ ]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Generating a map around toronto visulizing the borough aroud it

In [ ]:
map_toranto = folium.Map(location=[43.6532,-79.3832], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'],df['Longitude'],df['Borough'],df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toranto)  
    
map_toranto
Out[16]:


In [15]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [16]:
CLIENT_ID = 'VUN2QIYA32ZETHXDRJAPHZUPKLS2PR1IQKHLMMXGB0XYND0H' # your Foursquare ID
CLIENT_SECRET = 'GCF5MS34KXKVCXYQVZNAGZILJFJIZV0MS3213O3HRZNNHKVS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VUN2QIYA32ZETHXDRJAPHZUPKLS2PR1IQKHLMMXGB0XYND0H
CLIENT_SECRET:GCF5MS34KXKVCXYQVZNAGZILJFJIZV0MS3213O3HRZNNHKVS


In [17]:
print(df['Neighborhood'].values[0],df['Postalcode'].values[0])

Parkwoods M3A


In [18]:

print('latitude :',df['Latitude'].values[0])
print('longitude:',df['Longitude'].values[0])

latitude : 43.7532586
longitude: -79.3296565


analizing the neighborhood of 'parkwoods' under postal code 'M3A' with limits under 2 kilometers

In [19]:
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
 CLIENT_SECRET, 
    VERSION, 
    43.7532586,
-79.3296565, 
    2000,#two kilometer 
    100)

In [ ]:
results = requests.get(url).json()
print('there are {} famous venues near by'.format(results['response']['totalResults']))

Vsisulazing the venues nearby

In [ ]:

res2=results['response']['groups'][0]['items']
for res in res2:
    print(res['venue']['name'])
    print("     at a distance of---> ",res['venue']['location']['distance'])
print(' these are the 3 venues')

In [ ]:
map_parkwoods=folium.Map(location=[43.7532586,-79.3296565],zoom_start=13)
map_parkwoods

In [ ]:
df_clustered=pd.DataFrame(columns=['venue','latitude','longitude'])
for res in res2:
    latit=res['venue']['location']['lat']
    longit=res['venue']['location']['lng']
    label=res['venue']['name']
    df_clustered=df_clustered.append({'venue':label,'latitude':latit,'longitude':longit},ignore_index=True)
df_clustered.head(9)

clustering and visualizing

In [23]:
from sklearn.cluster import KMeans
print('import succesful')

import succesful


In [ ]:
kclusters=6
kmeans = KMeans(n_clusters=kclusters)
kmeans.fit(df_clustered[['latitude','longitude']])

In [ ]:
df_clustered['cluster']=kmeans.labels_

In [ ]:
df_clustered.head(9)

In [26]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [ ]:
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

map_clusters=folium.Map(location=[43.7532586,-79.3296565],zoom_start=13)

markers_colors = []
for lat, lon, poi, cluster in zip(df_clustered['latitude'], df_clustered['longitude'], df_clustered['venue'], df_clustered['cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters